<a href="https://colab.research.google.com/github/andreidore/aicrowd_blitz_may_2020/blob/master/pkhnd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb
!pip install pandas==0.25.3


In [2]:
!wget https://s3.eu-central-1.wasabisys.com/aicrowd-public-datasets/aicrowd_educational_pkhnd/data/public/test.csv
!wget https://s3.eu-central-1.wasabisys.com/aicrowd-public-datasets/aicrowd_educational_pkhnd/data/public/train.zip
!unzip train.zip

--2020-05-10 17:10:29--  https://s3.eu-central-1.wasabisys.com/aicrowd-public-datasets/aicrowd_educational_pkhnd/data/public/test.csv
Resolving s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)... 130.117.252.11, 130.117.252.12, 130.117.252.13, ...
Connecting to s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)|130.117.252.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 538694 (526K) [text/csv]
Saving to: ‘test.csv.1’

test.csv.1          100%[===================>] 526.07K  --.-KB/s    in 0.02s   

2020-05-10 17:10:29 (21.8 MB/s) - ‘test.csv.1’ saved [538694/538694]

--2020-05-10 17:10:31--  https://s3.eu-central-1.wasabisys.com/aicrowd-public-datasets/aicrowd_educational_pkhnd/data/public/train.zip
Resolving s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)... 130.117.252.11, 130.117.252.12, 130.117.252.13, ...
Connecting to s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)|130.117.252.11|:443... connected.
H

In [0]:
import os
import pandas as pd
import numpy as np
import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score

In [4]:
pd.__version__

'0.25.3'

In [0]:
train_data_path = "train.csv" #path where data is stored
train_data = pd.read_csv(train_data_path) #load data in dataframe using pandas

In [6]:
print(train_data.shape)
print(train_data.head())

(1000000, 11)
   S1  C1  S2  C2  S3  C3  S4  C4  S5  C5  label
0   1   1   1  13   2   4   2   3   1  12      0
1   3  12   3   2   3  11   4   5   2   5      1
2   1   9   4   6   1   4   3   2   3   9      1
3   1   4   3  13   2  13   2   1   3   6      1
4   3  10   2   7   1   2   2  11   4   9      0


In [7]:
pandas_profiling.ProfileReport(train_data)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,11
Number of observations,1000000
Total Missing (%),0.0%
Total size in memory,83.9 MiB
Average record size in memory,88.0 B
Numeric,11
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [10]:
X_train, X_val= train_test_split(train_data, test_size=0.2, random_state=42)
X_train,y_train = X_train.iloc[:,:-1],X_train.iloc[:,-1]
X_val,y_val = X_val.iloc[:,:-1],X_val.iloc[:,-1]
print(X_train)
print(y_train)
print(X_val)
print(y_val)

        S1  C1  S2  C2  S3  C3  S4  C4  S5  C5
566853   2  11   3   4   3   3   3  13   2   3
382311   2  11   1   5   2   9   4  13   3   5
241519   4   2   4   3   4   7   4  12   4   4
719220   1   8   4   5   2  13   2   6   4   8
905718   2   7   4   3   3   5   2  13   4  12
...     ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
259178   3   6   1   5   4  10   4   1   4  12
365838   1  10   3  13   2   8   1   1   2   1
131932   2   3   4   8   1   1   4   1   1   5
671155   3   5   1   8   3  12   4   6   4   8
121958   4   5   2  11   3   9   3   1   1   2

[800000 rows x 10 columns]
566853    1
382311    1
241519    5
719220    1
905718    0
         ..
259178    0
365838    1
131932    1
671155    1
121958    0
Name: label, Length: 800000, dtype: int64
        S1  C1  S2  C2  S3  C3  S4  C4  S5  C5
987231   1  12   4   9   3  12   3  13   2   4
79954    4   4   2   4   2  13   4  12   1   8
567130   4   3   4   7   1   8   2   3   3  12
500891   2   9   1   3   4   5   2   7   3   4

In [11]:
classifier = SVC(gamma='auto',max_iter=10)
classifier.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=10, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [13]:
y_pred = classifier.predict(X_val)
precision = precision_score(y_val,y_pred,average='micro')
recall = recall_score(y_val,y_pred,average='micro')
accuracy = accuracy_score(y_val,y_pred)
f1 = f1_score(y_val,y_pred,average='macro')
print("Accuracy of the model is :" ,accuracy)
print("Recall of the model is :" ,recall)
print("Precision of the model is :" ,precision)
print("F1 score of the model is :" ,f1)

Accuracy of the model is : 0.49894
Recall of the model is : 0.49894
Precision of the model is : 0.49894
F1 score of the model is : 0.07442511649369188


## Submission

In [0]:
final_test_path = "test.csv"
final_test = pd.read_csv(final_test_path)

submission = classifier.predict(final_test)

submission = pd.DataFrame(submission)
submission.to_csv('/tmp/submission.csv',header=['label'],index=False)
from google.colab import files
files.download('/tmp/submission.csv')